[Versión Interactiva](https://mybinder.org/v2/git/https%3A%2F%2Fgithub.com%2Fjuandadamo%2Fclases_fiuba/HEAD?filepath=6731%2FTP_1C_2021.ipynb)

In [1]:
!pip install numpy matplotlib ht CoolProp pandas IPython pint fluids

In [2]:
# -*- coding: utf-8 -*-
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import ht, pint, fluids
import CoolProp as cp
import pandas as pd
from IPython.display import Latex
ureg = pint.UnitRegistry();#(fmt_locale='es_ES')

### OBJETIVO

Diseño de un intercambiador de tipo casco y tubos para el enfriamiento de una corriente de un hidrocarburo liviano, utilizando como corriente de servicio, agua de torre de enfriamiento.

### ENUNCIADO

Se requiere enfriar una cantidad \(\dot m_C\) de un hidrocarburo liviano desde una temperatura \(T_{CE}\),
garantizando a la salida una temperatura \(T_{CS}\). Para ello se dispone de un caudal \(\dot m_F\) de agua de torre de enfriamiento, que ingresa a una temperatura de \(T_{FE}\). El agua no podrá calentarse por encima de
55°C. Considerar al agua como el fluido que pasa por los tubos y al hidrocarburo por la carcasa.
 

Se pide:

    1. Determinar las características de un intercambiador de casco y tubos que cumpla con los requerimientos. 
    2. Calcular las caídas de presión de ambas corrientes en su paso por el equipo.
    3. Completar el cuadro de resumen que contenga las características constructivas y de operación fundamentales calculadas (Entregar el archivo Excel completo adjunto).
    4. Hacer un plano conceptual a escala del equipo en el que debe estar presente la carcasa, tubos, bafles, cabezales, conexiones de entrada y salida de las corrientes, etc.
    5. Hacer un plano a escala de la placa portatubos y su relación con la carcasa.
    6. Durante la operación, el proceso sufre cierto desbalance que hace que la temperatura de entrada del hidrocarburo disminuya 5ºC. Manteniendo los caudales constantes, determinar las nuevas temperaturas de salida. ¿Verifican las condiciones de funcionamiento requeridas? (Se recomienda utilizar el método e-NUT)
    7. En los casos que no verifiquen, se dispone de un sistema de control del caudal de agua, para poder llevar al equipo dentro de los parámetros de funcionamiento adecuados. ¿Cuál es este nuevo caudal? (Se recomienda utilizar el método e-NUT)


DATOS

Corriente de Servicio: Agua de torre de enfriamiento tratada.

Corriente de Proceso: Hidrocarburo liviano

Temperatura del etilenglicol a la entrada – TFE [ºC] = 32.+0.5·UNP

Temperatura del etilenglicol a la salida – TFS [ºC] = 40

Temperatura del agua a la entrada – TCE [ºC] = 65

Temperatura del agua de salida mínima – TFSmin [ºC] = 20

Caudal de hidrocarburo liviano – mF [kg/s] (45+UNP)/10

Nota:
    • UNP= último número de padrón (alumnos de intercambio UNP=4)

In [3]:
Npadron = 73245
UNP = np.float(np.str(Npadron)[-1])

In [4]:
data = [['Corriente de Servicio: ','Agua de torre de enfriamiento tratada.'],
       ['Corriente de Proceso: ','Hidrocarburo liviano'],
       ['Temperatura del agua a la entrada – TFE [ºC] ',32.5+0.5*UNP],
       ['Temperatura máxima del agua a la salida – TFS max [ºC]',55],
        ['Temperatura de proceso a la entrada – TCE [ºC]',90],
        ['Temperatura de proceso a la salida – TCS [ºC]',45],
        ['Caudal de Hidrocarburo liviano',14-0.5*UNP]       
       ]
Datos = pd.DataFrame(data)
Datos_np = Datos.to_numpy()
display(Datos)

,0,1
0,Corriente de Servicio:,Agua de torre de enfriamiento tratada.
1,Corriente de Proceso:,Hidrocarburo liviano
2,Temperatura del agua a la entrada – TFE [ºC],35
3,Temperatura máxima del agua a la salida – TFS ...,55
4,Temperatura de proceso a la entrada – TCE [ºC],90
5,Temperatura de proceso a la salida – TCS [ºC],45
6,Caudal de Hidrocarburo liviano,11.5


In [5]:
################
#     DATOS    #
################

Tfe = (Datos_np[2,1]+273)*ureg.K
Tfs_max = (Datos_np[3,1]+273)*ureg.K
Tce = (Datos_np[4,1]+273)*ureg.K
Tcs = (Datos_np[5,1]+273)*ureg.K
m_f = Datos_np[6,1]*ureg.kg/ureg.second

In [6]:
# Cálculos 

Tfs = Tfs_max
Tfs = Tfs_max - 5*ureg.K

#Calor intercambiado

Tf_media = (Tfe+Tfs)/2
C_f = cp.CoolProp.PropsSI('C','T',Tf_media.magnitude,'P',101.325e3,'Water')*ureg.joule/ureg.kg/ureg.K
mu_f = cp.CoolProp.PropsSI('V','T',Tf_media.magnitude,'P',101.325e3,'Water')*ureg.kg/ureg.m/ureg.second
rho_f = cp.CoolProp.PropsSI('D','T',Tf_media.magnitude,'P',101.325e3,'Water')*ureg.kg*ureg.m**-3
nu_f = mu_f / rho_f
Pr_f = cp.CoolProp.PropsSI('Prandtl','T',Tf_media.magnitude,'P',101.325e3,'Water')
k_f = cp.CoolProp.PropsSI('L','T',Tf_media.magnitude,'P',101.325e3,'Water')*ureg.watt/ureg.m/ureg.K

Q = C_f*m_f*(Tfs-Tfe)


#Q = C_c*m_c*(Tcs-Tce)
Tc_media0 = (Tce+Tcs)/2
C_c = cp.CoolProp.PropsSI('C','T',Tc_media0.magnitude,'P',101.325e3,'Benzene')*ureg.joule/ureg.kg/ureg.K
mu_c = cp.CoolProp.PropsSI('V','T',Tc_media0.magnitude,'P',101.325e3,'Benzene')*ureg.kg/ureg.m/ureg.second
rho_c = cp.CoolProp.PropsSI('D','T',Tc_media0.magnitude,'P',101.325e3,'Benzene')*ureg.kg*ureg.m**-3
Pr_c = cp.CoolProp.PropsSI('Prandtl','T',Tc_media0.magnitude,'P',101.325e3,'Benzene')
k_c = cp.CoolProp.PropsSI('L','T',Tc_media0.magnitude,'P',101.325e3,'Benzene')*ureg.watt/ureg.m/ureg.K
nu_c = mu_c / rho_c



m_c = Q / C_c/(Tce-Tcs)

Cmin = np.min([(C_c*m_c).magnitude,(C_f*m_f).magnitude])
Cmax = np.max([(C_c*m_c).magnitude,(C_f*m_f).magnitude])

Cr = Cmin/Cmax

Qmax = Cmin*(Tce-Tfe) 
epsilon = Q.magnitude/Qmax.magnitude

dTlm = ht.LMTD(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs)

#  pasos de carcaza
Npasocarcasa = 1
Ft = ht.F_LMTD_Fakheri(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs, shells=Npasocarcasa)
UA = Q/(dTlm*Ft)
Nut = ht.NTU_from_effectiveness(effectiveness=epsilon, Cr=Cr, subtype='S&T')
Nut = ht.NTU_from_UA(UA,Cmin)

display(Latex(f'Calor Intercambiado: Q={Q.to(ureg.kW):.2f}'))
display(Latex(f'Nut = {Nut.magnitude:.2f}, '+'\(\\varepsilon=\)'+f' {epsilon:.2f}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

U \(\simeq\) 250-450 W/m\(^2\) K según: CAO, Eduardo. Heat transfer in process engineering. McGraw-Hill Education, 2010

In [7]:
# Suponemos U = 300
U = 250*ureg.watt/ureg.m**2/ureg.K
A = (UA / U).to_base_units()
display(Latex(f'Área = {A:~.2f}'))

#Suponiendo 
L_tubo = 6*ureg.m
NPS, Dint, Dext, esp_t = fluids.nearest_pipe(Do=1*24.5e-3, schedule='5S')
Ai_tubo = (Dint*ureg.m)**2*np.pi/4
Ae_tubo = L_tubo*(Dext*ureg.m)*2*np.pi
Npasotubos = 2

Ntubos = np.int( A.magnitude /( Npasotubos*Ae_tubo.magnitude))
display(Latex(f'Tubos de Nominal Pipe Size: {NPS:.1f}, \(D_{{ext}}={Dext*1e3:.1f}\)mm,  \(D_{{int}}={Dint*1e3:.1f}\)mm,              \(e_{{tubo}}={esp_t*1e3:.1f}\)mm')) 
display(Latex(f'Largo tubos = {L_tubo:~.2f}'))
Area_propuesta = Ntubos *  (Ae_tubo*Npasotubos)
display(Latex(f'Área propuesta = {Area_propuesta:~.2f}'))
display(Latex(f'Número de tubos = {Ntubos:0d}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
# Flujo por los tubos: agua

m_t = m_f

m_tubo = m_t / ( Ntubos )
g_tubo = m_tubo / rho_f
U_tubo = g_tubo / (Ai_tubo) 

Re_tubo = U_tubo * Dint*ureg.m / nu_f

# para estimar un paso en tubos a partir de un Re de diseño
#Npasotubos =  Re_design*(np.pi*Dint*ureg.m*Ntubos*mu_f ) / (4*m_t)
display(Latex('$Re_{{tubo}} = $' + f' {Re_tubo.magnitude:.0f}'))

<IPython.core.display.Latex object>

In [9]:
p1 = 1.25
Dcarcasa0 = ht.DBundle_for_Ntubes_Phadkeb(Ntubos, Dext, p1*Dext, Npasotubos, angle=45)

print('Diametro Carcasa = %.2f\'\''%(Dcarcasa0/25.4e-3))

Dcarcasa = (17+1/4)*25.4e-3 ##el valor mas cercano a Dcarcasa0
Do = Dext
pitch = p1*Dext


Ntubos = ht.Ntubes(Dcarcasa, Do, pitch, Ntp=Npasotubos, angle=90, Method=None)

print('Diametro Carcasa final = %.2f\'\''%(Dcarcasa/25.4e-3))
print('Numero de tubos = %0d'%(Ntubos))

A_real = Ae_tubo*Ntubos*Npasotubos
print(f'Areal real ={A_real:.2f}')


Diametro Carcasa = 17.04''
Diametro Carcasa final = 17.25''
Numero de tubos = 108
Areal real =217.42 meter ** 2


In [10]:
m_tubo = m_t / ( Ntubos )
g_tubo = m_tubo / rho_f
U_tubo = g_tubo / (Ai_tubo) 

Re_tubo = U_tubo * Dint*ureg.m / nu_f

# para estimar un paso en tubos a partir de un Re de diseño
#Npasotubos =  Re_design*(np.pi*Dint*ureg.m*Ntubos*mu_f ) / (4*m_t)
display(Latex('$Re_{{tubo}} = $' + f' {Re_tubo.magnitude:.0f}'))

Nu_f = ht.conv_internal.turbulent_Colburn(Re_tubo,Pr_f)
h_f = Nu_f*k_f/(Dint*ureg.m)

display(Latex('$h_f=$'+f'{h_f:.1f}'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [11]:
# En la carcasa

B = 0.25*Dcarcasa*ureg.m

#Pag 76 Cao, Dequivalente
Pt = pitch*ureg.m
c = (pitch-Dext)*ureg.m
As = Dcarcasa*ureg.m*c*B/Pt
Gs = m_c / As


Deq = 4*(pitch**2-np.pi*Dext**2/4) / (np.pi*Dext)*ureg.m
Aeq =  Deq**2*np.pi/4*Ntubos*Npasotubos/4



Re_c = Deq*m_c/mu_c/Aeq

Re_c = Deq*Gs/mu_c
Nu_c = 0.36*Re_c**0.55*Pr_c**0.3333

h_c = Nu_c*k_c/Deq

display(Latex('$h_c=$'+f'{h_c:.1f}'))


<IPython.core.display.Latex object>

In [12]:
# Temperatura de pared Tw

# (Tcm-Tw)*A*h_c = (Tw-Tfm) *A*h_f
#Tw*h_c+Tw*h_f = Tcm*h_c + Tfm*h_f

Tw = (h_c*Tc_media0+h_f*Tf_media) / (h_c+h_f )
#con este valor se corrigen las propiedades de los fluidos

Tcmedia = (Tc_media0+Tw)/2
Tfmedia = (Tf_media+Tw)/2

Pr_f = cp.CoolProp.PropsSI('PRANDTL','T',Tfmedia.magnitude,'P',101.325e3,'Water')
k_f = cp.CoolProp.PropsSI('L','T',Tfmedia.magnitude,'P',101.325e3,'Water')*ureg.watt*ureg.m**-1*ureg.K**-1
mu_f = cp.CoolProp.PropsSI('V','T',Tfmedia.magnitude,'P',101.325e3,'Water')*ureg.kg*ureg.m**-1*ureg.second**-1

Pr_c = cp.CoolProp.PropsSI('PRANDTL','T',Tcmedia.magnitude,'P',101.325e3,'Benzene')
k_c = cp.CoolProp.PropsSI('L','T',Tcmedia.magnitude,'P',101.325e3,'Benzene')*ureg.watt*ureg.m**-1*ureg.K**-1
mu_c = cp.CoolProp.PropsSI('V','T',Tcmedia.magnitude,'P',101.325e3,'Benzene')*ureg.kg*ureg.m**-1*ureg.second**-1
rho_c = cp.CoolProp.PropsSI('D','T',Tcmedia.magnitude,'P',101.325e3,'Benzene')*ureg.kg*ureg.m**-3
rho_f = cp.CoolProp.PropsSI('D','T',Tfmedia.magnitude,'P',101.325e3,'Water')*ureg.kg*ureg.m**-3

mu_w = cp.CoolProp.PropsSI('V','T',Tw.magnitude,'P',101.325e3,'Benzene')*ureg.kg*ureg.m**-1*ureg.second**-1


In [13]:
Re_c = Deq*m_c/mu_c/Aeq
U_carcasa = m_c/Aeq/rho_c
Re_c = Deq*Gs/mu_c
Nu_c = 0.36*Re_c**0.55*Pr_c**0.3333

h_c = Nu_c*k_c/Deq

Nu_f = ht.conv_internal.turbulent_Colburn(Re_tubo.magnitude,Pr_f)
h_f = Nu_f*k_f/(Dint*ureg.m)
display(Latex('$h_f=$'+f'{h_f:.1f}'))
display(Latex('$h_c=$'+f'{h_c:.1f}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [14]:
U_carcasa,U_tubo

(0.3481335296518343 <Unit('meter / second')>,
 0.24977480094480684 <Unit('meter / second')>)

In [15]:
Rf_f = (0.86*0.4e-4 )*ureg.K*ureg.m**2*ureg.watt**-1
Rf_c =  0.86*0.75e-4*ureg.K*ureg.m**2*ureg.watt**-1
Rf = Rf_f+Rf_c
Ulimpio = 1 / (Dext/Dint/h_c+1/h_f )
U = 1 / (1/Ulimpio + Rf )#*(1+Dext/Dint))



#A_calculo = Q / (U*dTlm*Ft)

display(Latex('$U_{limpio}=$'+f'{Ulimpio:.1f}'))
display(Latex('$U_{final}=$'+f'{U:.1f}'))

#display(Latex('$A_{final}=$'+f'{A_calculo.to_base_units():.1f}'))#



#print('A real inicial %.2f'%A_real)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [16]:
# Caida de presion

# Calculo para flujo en tubos
roughness = fluids.roughness_Farshad('Carbon steel, bare', D=Dint)

f_c = fluids.friction.friction_factor(Re=Re_tubo, eD=roughness/Dint)

f_c2 = 1.2*(0.0014+0.125/Re_tubo**.32)

g_tubo = m_tubo / rho_f
U_tubo = g_tubo / (Ai_tubo) 

Delta_p_friccion = 0.5*rho_f*U_tubo**2*L_tubo/(Dint*ureg.m)

Delta_p_cabezales = 4*Npasotubos*U_tubo**2*rho_f/2

Delta_p_tubos = Delta_p_cabezales+Delta_p_friccion

display(Latex('Factor friccion tubos, $f_f=$'+f'{f_c.magnitude:.2g}'))
display(Latex('Perdida friccion tubos, $\Delta p_f=$'+f'{Delta_p_friccion.to(ureg.kPa):.2g}'))
display(Latex('Perdida cabezales, $\Delta p_c=$'+f'{Delta_p_cabezales.to(ureg.kPa):.2g}'))
display(Latex('Perdida tubos, $\Delta p_t=\Delta p_c+\Delta p_f= $'+f'{Delta_p_tubos.to(ureg.kPa):.2g}'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [17]:

LSpacing = Dcarcasa
NBaffles = np.floor(L_tubo.magnitude / LSpacing)

Delta_p_carcasa = ht.conv_tube_bank.dP_Kern(m_c.magnitude, rho_c.magnitude, mu_c.magnitude, Dcarcasa, LSpacing, pitch, Dext, NBaffles, mu_w=mu_w.magnitude)*ureg.Pa

display(Latex('Perdida flujo en Carcasa, $\Delta p_{Carcasa}=$'+f'{Delta_p_carcasa.to(ureg.kPa):.1f}'))

<IPython.core.display.Latex object>

In [18]:
data_excel = pd.read_excel('resultados_fmt.xls')

In [19]:
comportamiento_unidad = pd.DataFrame(np.zeros((29,3),dtype='str'),columns=[' ','Lado Carcasa','Lado Tubos'])
comportamiento_unidad.loc[0:30,' ']  = data_excel.to_numpy()[5:34,0]


In [20]:
comportamiento_unidad.loc[0,1:3] = ['Benceno','Agua']
comportamiento_unidad.loc[1,1:3] = [f'{k_c:~.3f}',f'{k_f:~.3f}']
comportamiento_unidad.loc[2,1:3] = [f'{rho_c:~.0f}',f'{rho_f:~.0f}']
comportamiento_unidad.loc[3,1:3] = [f'{C_c:~.0f}',f'{C_f:~.0f}']
comportamiento_unidad.loc[4,1:3] = [f'{mu_c:~.2e}',f'{mu_f:~.2e}']
comportamiento_unidad.loc[5,1:3] = [f'{Tce.to(ureg.degC):~.0f}',f'{Tfe.to(ureg.degC):~.0f}']
comportamiento_unidad.loc[6,1:3] = [f'{Tcs.to(ureg.degC):~.0f}',f'{Tfs.to(ureg.degC):~.0f}']
comportamiento_unidad.loc[7,1:3] = [f'{Npasocarcasa:.0f}',f'{Npasotubos:.0f}']
comportamiento_unidad.loc[8,1:3] = [f'{U_carcasa:~.2f}',f'{U_tubo:~.2f}']
comportamiento_unidad.loc[9,1:3] = [f'{Delta_p_carcasa.to(ureg.kPa):~.2f}',f'{Delta_p_tubos.to(ureg.kPa):~.2f}']
comportamiento_unidad.loc[10,1:3] = [f'{Rf_c:~.2e}',f'{Rf_f:~.2e}']

comportamiento_unidad.loc[13,1:3] = [f'{Ft.magnitude:.2f}','']
comportamiento_unidad.loc[14,1:3] = [f'{dTlm.to(ureg.delta_degC):~.2f}','']
comportamiento_unidad.loc[15,1:3] = [f'{Q.to(ureg.kW):~.1f}','']
comportamiento_unidad.loc[16,1:3] = [f'{Ulimpio:~.1f}','']
comportamiento_unidad.loc[17,1:3] = [f'{U:~.1f}','']
comportamiento_unidad.loc[19,1:3] = [f'{Dext*1e3*ureg.mm:~.1f}','']
comportamiento_unidad.loc[21,1:3] = [f'{Dint*1e3*ureg.mm:~.1f}','']
comportamiento_unidad.loc[20,1:3] = [f'{esp_t*1e3*ureg.mm:~.1f}','']
comportamiento_unidad.loc[22,1:3] = [f'{pitch*1e3*ureg.mm:~.1f}','']
comportamiento_unidad.loc[23,1:3] = [f'{Ntubos:.0f}','']
comportamiento_unidad.loc[24,1:3] = [f'{L_tubo:~.1f}','']
comportamiento_unidad.loc[25,1:3] = [f'{A_real:~.1f}','']
comportamiento_unidad.loc[26,1:3] = [f'{Dcarcasa*1e3*ureg.mm:~.0f}','']
comportamiento_unidad.loc[27,1:3] = [f'{LSpacing*1e3*ureg.mm:~.0f}','']
comportamiento_unidad.loc[28,1:3] = [f'{NBaffles:.0f}','']

In [21]:
comportamiento_unidad

,,Lado Carcasa,Lado Tubos
0,FLUIDOS,Benceno,Agua
1,CONDUCTIVIDAD TERMICA [W/m·K],0.129 W / K / m,0.639 W / K / m
2,DENSIDAD [kg/m3],835 kg / m ** 3,989 kg / m ** 3
3,CAPACIDAD CALORIFICA [J/kg·K],1862 J / K / kg,4180 J / K / kg
4,VISCOSIDAD DINAMICA [kg/m·s],3.89e-04 kg / m / s,5.63e-04 kg / m / s
5,TEMPERATURA DE ENTRADA [ºC],90 °C,35 °C
6,TEMPERATURA DE SALIDA [ºC],45 °C,50 °C
7,NUMERO DE PASOS [-],1,2
8,VELOCIDAD [m/s],0.35 m / s,0.25 m / s
9,CAIDA DE PRESION [kPa],2.03 kPa,8.20 kPa


Para conocer las entradas  http://www.coolprop.org/coolprop/HighLevelAPI.html#parameter-table